<a href="https://colab.research.google.com/github/kh9yaz/124Rus/blob/master/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Условная оптимизация

## Метод штрафных функций

В предыдущей работе нам было необходимо не только решить задачу оптимизации, но и учесть ораничения на возможные решения.

Как вы могли заметить, не все методы оптимизации из тех, которые доступны в пакете SciPy, поддерживают условную оптимизацию, при этом для некоторых задач одни методы более предпочтительны, чем другие, и может так оказаться, что для решения именно вашей задачи тот метод, который поддерживает условную оптимизацию, неприменим, а ограничения все-таки есть.

В этом случае на помощь приходят методы штрафных функций и барьерных поверхностей.

Основная идея у этих методов очень похожа, и заключается в том, что ограничения включаются прямо в оптимизируемую функцию, и делают ее тем более неоптимальной, чем дальше решение заходит в запрещенную область.

Разница между этими методами состоит в том, где находится исходная точка, с которой начинается поиск оптимального решения. В случае метода штрафных функций начальная точка не обязана находиться внутри зоны допустимых с точки зрения ограничений решений, а в случае барьерных функций - должна обязательно. Связано это с тем, что барьерные функции за пределами границы вновь уменьшаются, а штрафные функции проболжают возрастать тем больше, чем дальше за пределы границы вышла текущая точка.

Например, необходимо найти минимум функции $f(x) = (x - 2)^2 + 1$.

Мы знаем, что минимум данной функции находится в точке  $x = 2$, и значение минимизируемой функции в ней равно $1$.

Допустим, что существует ограничение $-\infty x \le 1$. В этом случае минимум функции будет как раз в точке 1.

Попробуем ввести барьерную функцию. Требования к ней просты: в допустимой зоне ее значение должно быть близким к нулю, чтобы не оказывать влияния на целевую функцию, а в запрещенной зоне ее значение должно увеличиваться.

Введем функцию $g(x) = \frac {1} {1 - x}$. Ее особенность состоит в том, что внутри целевого диапазона она достаточно мала, а при приближении к границе (число 1), начинает возрастать (поскольку $\frac {1} / {1 - 1}$ становится равной бесконечности).

Таким образом, целевая функция теперь приобретает вот такой вид:

$$
f(x) = (x - 2)^2 + 1 + \frac {1} {1 - x}
$$

Казалось бы, все хорошо? На самом деле, нет.

Дело в том, что для данного случая проблема заключается вот в чем: минимум нашей фукнции находится в точке $x = 1$ (мы помним, что на смом деле в точке $x = 2$, но у нас есть ограничение $-\infty x \le 1$, соответственно, минимум будет именно в точке $x = 1$). Однако, если мы используем ограничения...

In [ ]:
import numpy as np  # нам понадобится пакет numpy, чтобы использовать значение бесконечность (np.inf)
import math  #
from scipy.optimize import minimize  # берем готовую библиотечную функцию minimize
from scipy.optimize import LinearConstraint  # будем использовать линейные ограничения

In [ ]:
# описываем функцию, которую хотим минимизировать
def func_to_minimize(x):
    return (x - 2) * (x - 2) + 1 + 1 / (1 - x)

# Зададим начальное значение x
x = 0  # подумайте, почему начально значение в данном случае равно трем

# а теперь минимизируем нашу функцию
result = minimize(func_to_minimize, x)
print(result)  # выведем наш результа на экран

      fun: 6.0
 hess_inv: array([[1]])
      jac: array([-3.])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 174
      nit: 0
     njev: 83
   status: 2
  success: False
        x: array([0.])


... то окажется, что минимум оказался в точке $0$. Не совсем тот результат, что мы ожидали.

Исправить положение можно либо изменив барьерную функцию так, чтобы она возрастала очень резко по мере приближения к границе запретной зоны, либо добавив специальный коэффициент $\alpha$, снизив влияние барьерной функции:

$$
f(x) = (x - 2)^2 + 1 + \alpha \cdot \frac {1} {1 - x}
$$

Попробуем:

In [ ]:
alpha = 0.1

# описываем функцию, которую хотим минимизировать
def func_to_minimize(x):
    return (x - 2) * (x - 2) + 1 + alpha * 1 / (1 - x)

# Зададим начальное значение x
x = 0  # подумайте, почему начально значение в данном случае равно трем

# а теперь минимизируем нашу функцию
result = minimize(func_to_minimize, x)
print(result)  # выведем наш результа на экран

      fun: 1.115695243467632
 hess_inv: array([[0.15946138]])
      jac: array([1.47203085])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 199
      nit: 2
     njev: 97
   status: 2
  success: False
        x: array([1.17921244])


Как видим, результат оказался гораздо лучше, хоть и неидеальным.

## Задание для самостоятельной работы

Изменить код в ячейке ниже (код основан на предыдущей работе) таким образом, чтобы учитывлось ограничение на время движения (необходимо затратить не более 0.028 часа), и расход топлива был минимальным, при условии использования штрафных или барьерных функций

In [2]:
import math  #
from scipy.optimize import minimize  # берем готовую библиотечную функцию minimize

# задаим исходные данные в километрах

h1 = 0.10  # 100 метров
h2 = 0.10  # 100 метров
l = 1  # 1000 метров

v1 = 40  # 40 км/ч
v2 = 30  # 30 км/ч

c1 = 0.115  # л/км
c2 = 0.15  # л/км

def cons_t(x): #Функция для расчета времени
  t1 = math.sqrt(h1 * h1 + x * x) / v1
  t2 = math.sqrt(h2 * h2 + (l - x) * (l - x)) / v2
  return t1 + t2

# Штрафная функция для учета ограничений
def penalty_function(x):
    penalty = 0 # Если время превышает 0.028, добавляем штраф
    if cons_t(x) > 0.028:
        penalty = (cons_t(x) - 0.028) ** 2  # Квадрат штрафа
    return penalty

# описываем функцию, которую хотим минимизировать

def func_to_minimize(x):
  u1 = math.sqrt(h1 * h1 + x * x) * c1
  u2 = math.sqrt(h2 * h2 + (l - x) * (l - x)) * c2
  return u1 + u2 + penalty_function(x) # вставьте вашу функцию здесь

# Зададим начальное значение x, пусть это будет самое начало поля, т.е., вначале мы поедем вериткально вверх,
# а затем, как только достигнем конца засеянной части, кратчайшим путем поедем в правый верхний угол поля
x = 0

# а теперь минимизируем нашу функцию
result = minimize(func_to_minimize, x, method='trust-constr')

if result.success:
    print("Оптимальное значение x:", result.x[0])
    print("Минимальные затраты на топливо:", result.fun)
else:
    print("Минимизация не удалась:", result.message)

Оптимальное значение x: 0.8824076942120102
Минимальные затраты на топливо: 0.1252808893080318


<ipython-input-2-8243a7e11788>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u1 = math.sqrt(h1 * h1 + x * x) * c1
<ipython-input-2-8243a7e11788>:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u2 = math.sqrt(h2 * h2 + (l - x) * (l - x)) * c2
<ipython-input-2-8243a7e11788>:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t1 = math.sqrt(h1 * h1 + x * x) / v1
<ipython-input-2-8243a7e11788>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar